In [123]:
import os
import pandas as pd
import numpy as np

path = 'test-runs/test-run-20220722-0845 (complete)/'
output_path = 'test-runs/test-runs-some-nans-filled/'
final_labels = ['RWP0004 Max Height: 0.8  km Speed (m/s)',
                'RWP0004 Max Height: 0.8  km Direction (var)',
                'RWP0004 Max Height: 1.5  km Speed (m/s)',
                'RWP0004 Max Height: 1.5  km Direction (var)',
                'RWP0004 Max Height: 10  km Speed (m/s)',
                'RWP0004 Max Height: 10  km Direction (var)',
                'RWP0005 Max Height: 0.8  km Speed (m/s)',
                'RWP0005 Max Height: 0.8  km Direction (var)',
                'RWP0005 Max Height: 1.5  km Speed (m/s)',
                'RWP0005 Max Height: 1.5  km Direction (var)',
                'RWP0005 Max Height: 10  km Speed (m/s)',
                'RWP0005 Max Height: 10  km Direction (var)',
                'RWP0001 Max Height: 0.8  km Speed (m/s)',
                'RWP0001 Max Height: 0.8  km Direction (var)',
                'RWP0001 Max Height: 1.5  km Speed (m/s)',
                'RWP0001 Max Height: 1.5  km Direction (var)',
                'RWP0001 Max Height: 10  km Speed (m/s)',
                'RWP0001 Max Height: 10  km Direction (var)',
                'RWP0002 Max Height: 0.8  km Speed (m/s)',
                'RWP0002 Max Height: 0.8  km Direction (var)',
                'RWP0002 Max Height: 1.5  km Speed (m/s)',
                'RWP0002 Max Height: 1.5  km Direction (var)',
                'RWP0002 Max Height: 10  km Speed (m/s)',
                'RWP0002 Max Height: 10  km Direction (var)',
                'RWP0003 Max Height: 0.8  km Speed (m/s)',
                'RWP0003 Max Height: 0.8  km Direction (var)',
                'RWP0003 Max Height: 1.5  km Speed (m/s)',
                'RWP0003 Max Height: 1.5  km Direction (var)',
                'RWP0003 Max Height: 10  km Speed (m/s)',
                'RWP0003 Max Height: 10  km Direction (var)']

good_df_list_speed=[]
good_df_list_dir=[]
for f in os.listdir(path):
    df=pd.read_csv(path+'/'+f)
    df_set=df.loc[:,final_labels]
    factor='Speed'
    speed_columns=[]
    for c in df_set.columns:
        if factor in c:
            speed_columns.append(c)
    df_speed=df_set.loc[:,speed_columns]
    averages=df_speed.mean(axis=1)
    df[speed_columns]=df[speed_columns].T.fillna(averages).T
    if df[speed_columns].isna().sum().sum()==0:
        good_df_list_speed.append(df)
    factor='Direction'
    df_set=df.loc[:,final_labels]
    dir_columns=[]
    for c in df_set.columns:
        if factor in c:
            dir_columns.append(c)
    df_dir=df_set.loc[:,dir_columns]
    averages=df_dir.mean(axis=1)
    df[dir_columns]=df[dir_columns].T.fillna(averages).T
    if df[dir_columns].isna().sum().sum()==0:
        good_df_list_dir.append(df)
    df.to_csv(output_path+f)

In [124]:
good_df_list_speed_launch=[]
good_df_list_speed_scrub=[]
for df in good_df_list_speed:
    if df.iloc[0,-1] == 0:
        good_df_list_speed_launch.append(df)
    else:
        good_df_list_speed_scrub.append(df)
good_df_list_dir_launch=[]
good_df_list_dir_scrub=[]
for df in good_df_list_dir:
    if df.iloc[0,-1] == 0:
        good_df_list_dir_launch.append(df)
    else:
        good_df_list_dir_scrub.append(df)

In [125]:
speed_launch_avg=pd.concat(good_df_list_speed_launch)[speed_columns].mean(axis=1)
speed_scrub_avg=pd.concat(good_df_list_speed_scrub)[speed_columns].mean(axis=1)
dir_launch_avg=pd.concat(good_df_list_dir_launch)[dir_columns].mean(axis=1)
dir_scrub_avg=pd.concat(good_df_list_dir_scrub)[dir_columns].mean(axis=1)

In [ ]:
for f in os.listdir(output_path):
    df=pd.read_csv(path+'/'+f)
    df_speed=df.loc[:,speed_columns]
    if df[speed_columns].isna().sum().sum()>0:
        if df.iloc[0,-1]==0:
            df[speed_columns]=df[speed_columns].T.fillna(speed_launch_avg).T
        else:
            df[speed_columns]=df[speed_columns].T.fillna(speed_scrub_avg).T

    df_dir=df.loc[:,dir_columns]
    if df[dir_columns].isna().sum().sum()>0:
        if df.iloc[0,-1]==0:
            df[dir_columns]=df[dir_columns].T.fillna(dir_launch_avg).T
        else:
            df[dir_columns]=df[dir_columns].T.fillna(dir_scrub_avg).T
    print(df[dir_columns].isna().sum().sum())
    df.to_csv(output_path+f)